In [116]:
from utils import create_model_hash, get_model_and_tokenizer, create_model_hash, create_gradient, create_gradient_hash
from hparams import batch_size, sequence_length, topk_percent

model, tokenizer = get_model_and_tokenizer()
print (create_model_hash(model))

d727b1a528ac34b043477f919fa3abc7fcf2e811ba84329c03ab3fe1e0366fb3


In [117]:
gradient = create_gradient(
    model = model,
    tokenizer = tokenizer,
    pages = [1],
    batch_size = batch_size,
    sequence_length = sequence_length,
    device = 'cpu',
    topk_percent = topk_percent
)
print( create_gradient_hash( gradient ) )

c3744a9e53943091c082b78e997b27a2cf9dcd2d0522a727b929acf482e764a8
